# Model Selection

This Notebook tests multiple regression models in their base configs to select the most promising algorithm for the task


In [1]:
import os
import polars as pl
from sklearn.preprocessing import (
    StandardScaler,
    PolynomialFeatures,
)
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, root_mean_squared_error as rmse
import mlflow
import mlflow.sklearn
import mlflow.xgboost

In [2]:
def get_data(poly_features: int = 1):
    # Get Data
    data = pl.read_parquet("data.parquet")
    data = data.drop(["Step", "Light_ID", "Lane", "Intersection_u", "Sim_ID"])
    data = data.with_columns(pl.col("Is_Entrypoint").cast(pl.Int8))
    print(f"Data: {data.shape}")
    print(f"{data.collect_schema()}")

    # Split Data
    X = data.drop("Num_Cars").to_numpy()
    y = data.select(pl.col("Num_Cars")).to_numpy()
    y = y.ravel()
    print("")
    print(f"X: {X.shape}")
    print(f"y: {y.shape}")

    # Scale
    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    # Polynomial Features
    if poly_features > 1:
        poly = PolynomialFeatures(degree=poly_features)
        X = poly.fit_transform(X)

    # Train Test Split
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, train_size=0.6, test_size=0.4, random_state=42
    )
    print("")
    print(f"X_train: {X_train.shape}")
    print(f"X_test: {X_test.shape}")
    print(f"y_train: {y_train.shape}")
    print(f"y_test {y_test.shape}")

    # Train Test Validation Split
    X_test, X_val, y_test, y_val = train_test_split(
        X_test, y_test, train_size=0.5, test_size=0.5, random_state=42
    )
    print("")
    print(f"X_test: {X_test.shape}")
    print(f"X_val: {X_val.shape}")
    print(f"y_test: {y_test.shape}")
    print(f"y_val: {y_val.shape}")

    return X_train, X_test, X_val, y_train, y_test, y_val

## Linear Regression


In [13]:
from sklearn.linear_model import LinearRegression

In [14]:
X_train, X_test, X_val, y_train, y_test, y_val = get_data()

Data: (6610000, 5)
Schema({'Time': Int16, 'Num_Cars': Int16, 'Centrality': Float32, 'Is_Entrypoint': Int8, 'Distance': Int16})

X: (6610000, 4)
y: (6610000,)

X_train: (3966000, 4)
X_test: (2644000, 4)
y_train: (3966000,)
y_test (2644000,)

X_test: (1322000, 4)
X_val: (1322000, 4)
y_test: (1322000,)
y_val: (1322000,)


In [15]:
with mlflow.start_run():
    mlflow.log_param("model_type", "LinearRegression")

    model = LinearRegression(n_jobs=-1)
    model.fit(X_train, y_train)

    print(model)
    print("")

    y_pred_train = model.predict(X_train)
    train_r2 = r2_score(y_true=y_train, y_pred=y_pred_train)
    train_rmse = rmse(y_true=y_train, y_pred=y_pred_train)

    print(f"R2 Score (Training Data): {train_r2}")
    print(f"RMSE (Training Data): {train_rmse}")

    mlflow.log_metric("train_r2", train_r2)
    mlflow.log_metric("train_rmse", train_rmse)

    y_pred_test = model.predict(X_test)
    test_r2 = r2_score(y_true=y_test, y_pred=y_pred_test)
    test_rmse = rmse(y_true=y_test, y_pred=y_pred_test)

    print(f"R2 Score (Test Data): {test_r2}")
    print(f"RMSE (Test Data): {test_rmse}")

    mlflow.log_metric("test_r2", test_r2)
    mlflow.log_metric("test_rmse", test_rmse)

    mlflow.sklearn.log_model(model, "model")

LinearRegression(n_jobs=-1)

R2 Score (Training Data): 0.09408396482467651
RMSE (Training Data): 1.4932094812393188
R2 Score (Test Data): 0.09340804815292358
RMSE (Test Data): 1.4924378395080566


2025/04/13 15:36:28 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


## Linear Regression with Polynomial Features


In [7]:
from sklearn.linear_model import LinearRegression

In [16]:
X_train, X_test, X_val, y_train, y_test, y_val = get_data(poly_features=2)

Data: (6610000, 5)
Schema({'Time': Int16, 'Num_Cars': Int16, 'Centrality': Float32, 'Is_Entrypoint': Int8, 'Distance': Int16})

X: (6610000, 4)
y: (6610000,)

X_train: (3966000, 15)
X_test: (2644000, 15)
y_train: (3966000,)
y_test (2644000,)

X_test: (1322000, 15)
X_val: (1322000, 15)
y_test: (1322000,)
y_val: (1322000,)


In [17]:
with mlflow.start_run():
    mlflow.log_param("model_type", "LinearRegression")
    mlflow.log_param("degree", 2)

    model = LinearRegression(n_jobs=-1)
    model.fit(X_train, y_train)

    print(model)
    print("")

    y_pred_train = model.predict(X_train)
    train_r2 = r2_score(y_true=y_train, y_pred=y_pred_train)
    train_rmse = rmse(y_true=y_train, y_pred=y_pred_train)

    print(f"R2 Score (Training Data): {train_r2}")
    print(f"RMSE (Training Data): {train_rmse}")

    mlflow.log_metric("train_r2", train_r2)
    mlflow.log_metric("train_rmse", train_rmse)

    y_pred_test = model.predict(X_test)
    test_r2 = r2_score(y_true=y_test, y_pred=y_pred_test)
    test_rmse = rmse(y_true=y_test, y_pred=y_pred_test)

    print(f"R2 Score (Test Data): {test_r2}")
    print(f"RMSE (Test Data): {test_rmse}")

    mlflow.log_metric("test_r2", test_r2)
    mlflow.log_metric("test_rmse", test_rmse)

    mlflow.sklearn.log_model(model, "model")

LinearRegression(n_jobs=-1)

R2 Score (Training Data): 3.9517879486083984e-05
RMSE (Training Data): 1.5688021183013916
R2 Score (Test Data): 5.6743621826171875e-05
RMSE (Test Data): 1.5673933029174805


2025/04/13 15:36:47 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [18]:
X_train, X_test, X_val, y_train, y_test, y_val = get_data(poly_features=3)

Data: (6610000, 5)
Schema({'Time': Int16, 'Num_Cars': Int16, 'Centrality': Float32, 'Is_Entrypoint': Int8, 'Distance': Int16})

X: (6610000, 4)
y: (6610000,)

X_train: (3966000, 35)
X_test: (2644000, 35)
y_train: (3966000,)
y_test (2644000,)

X_test: (1322000, 35)
X_val: (1322000, 35)
y_test: (1322000,)
y_val: (1322000,)


In [19]:
with mlflow.start_run():
    mlflow.log_param("model_type", "LinearRegression")
    mlflow.log_param("degree", 3)

    model = LinearRegression(n_jobs=-1)
    model.fit(X_train, y_train)

    print(model)
    print("")

    y_pred_train = model.predict(X_train)
    train_r2 = r2_score(y_true=y_train, y_pred=y_pred_train)
    train_rmse = rmse(y_true=y_train, y_pred=y_pred_train)

    print(f"R2 Score (Training Data): {train_r2}")
    print(f"RMSE (Training Data): {train_rmse}")

    mlflow.log_metric("train_r2", train_r2)
    mlflow.log_metric("train_rmse", train_rmse)

    y_pred_test = model.predict(X_test)
    test_r2 = r2_score(y_true=y_test, y_pred=y_pred_test)
    test_rmse = rmse(y_true=y_test, y_pred=y_pred_test)

    print(f"R2 Score (Test Data): {test_r2}")
    print(f"RMSE (Test Data): {test_rmse}")

    mlflow.log_metric("test_r2", test_r2)
    mlflow.log_metric("test_rmse", test_rmse)

    mlflow.sklearn.log_model(model, "model")

LinearRegression(n_jobs=-1)

R2 Score (Training Data): 3.9517879486083984e-05
RMSE (Training Data): 1.5688021183013916
R2 Score (Test Data): 5.6862831115722656e-05
RMSE (Test Data): 1.567393183708191


2025/04/13 15:37:00 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


## Decision Tree Regression


In [20]:
from sklearn.tree import DecisionTreeRegressor

In [21]:
X_train, X_test, X_val, y_train, y_test, y_val = get_data()

Data: (6610000, 5)
Schema({'Time': Int16, 'Num_Cars': Int16, 'Centrality': Float32, 'Is_Entrypoint': Int8, 'Distance': Int16})

X: (6610000, 4)
y: (6610000,)

X_train: (3966000, 4)
X_test: (2644000, 4)
y_train: (3966000,)
y_test (2644000,)

X_test: (1322000, 4)
X_val: (1322000, 4)
y_test: (1322000,)
y_val: (1322000,)


In [22]:
with mlflow.start_run():
    mlflow.log_param("model_type", "DecisionTreeRegressor")

    model = DecisionTreeRegressor(random_state=42)
    model.fit(X_train, y_train)

    print(model)
    print("")

    y_pred_train = model.predict(X_train)
    train_r2 = r2_score(y_true=y_train, y_pred=y_pred_train)
    train_rmse = rmse(y_true=y_train, y_pred=y_pred_train)

    print(f"R2 Score (Training Data): {train_r2}")
    print(f"RMSE (Training Data): {train_rmse}")

    mlflow.log_metric("train_r2", train_r2)
    mlflow.log_metric("train_rmse", train_rmse)

    y_pred_test = model.predict(X_test)
    test_r2 = r2_score(y_true=y_test, y_pred=y_pred_test)
    test_rmse = rmse(y_true=y_test, y_pred=y_pred_test)

    print(f"R2 Score (Test Data): {test_r2}")
    print(f"RMSE (Test Data): {test_rmse}")

    mlflow.log_metric("test_r2", test_r2)
    mlflow.log_metric("test_rmse", test_rmse)

    mlflow.sklearn.log_model(model, "model")

DecisionTreeRegressor(random_state=42)

R2 Score (Training Data): 0.46521877733411565
RMSE (Training Data): 1.1472678598443036
R2 Score (Test Data): 0.02001084905453221
RMSE (Test Data): 1.5516754321690063


2025/04/13 15:37:51 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


## Random Forest Regression


In [23]:
from sklearn.ensemble import RandomForestRegressor

In [24]:
X_train, X_test, X_val, y_train, y_test, y_val = get_data()

Data: (6610000, 5)
Schema({'Time': Int16, 'Num_Cars': Int16, 'Centrality': Float32, 'Is_Entrypoint': Int8, 'Distance': Int16})

X: (6610000, 4)
y: (6610000,)

X_train: (3966000, 4)
X_test: (2644000, 4)
y_train: (3966000,)
y_test (2644000,)

X_test: (1322000, 4)
X_val: (1322000, 4)
y_test: (1322000,)
y_val: (1322000,)


In [25]:
with mlflow.start_run():
    mlflow.log_param("model_type", "RandomForestRegressor")

    model = RandomForestRegressor(random_state=42)
    model.fit(X_train, y_train)

    print(model)
    print("")

    y_pred_train = model.predict(X_train)
    train_r2 = r2_score(y_true=y_train, y_pred=y_pred_train)
    train_rmse = rmse(y_true=y_train, y_pred=y_pred_train)

    print(f"R2 Score (Training Data): {train_r2}")
    print(f"RMSE (Training Data): {train_rmse}")

    mlflow.log_metric("train_r2", train_r2)
    mlflow.log_metric("train_rmse", train_rmse)

    y_pred_test = model.predict(X_test)
    test_r2 = r2_score(y_true=y_test, y_pred=y_pred_test)
    test_rmse = rmse(y_true=y_test, y_pred=y_pred_test)

    print(f"R2 Score (Test Data): {test_r2}")
    print(f"RMSE (Test Data): {test_rmse}")

    mlflow.log_metric("test_r2", test_r2)
    mlflow.log_metric("test_rmse", test_rmse)

    mlflow.sklearn.log_model(model, "model")

RandomForestRegressor(random_state=42)

R2 Score (Training Data): 0.4614106202046152
RMSE (Training Data): 1.1513454394562874
R2 Score (Test Data): 0.08627984488903662
RMSE (Test Data): 1.4982933517121462


2025/04/13 15:52:50 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


## Gradient Boosting Regression


In [3]:
from xgboost import XGBRegressor

In [4]:
X_train, X_test, X_val, y_train, y_test, y_val = get_data()

Data: (6610000, 5)
Schema({'Time': Int16, 'Num_Cars': Int16, 'Centrality': Float32, 'Is_Entrypoint': Int8, 'Distance': Int16})

X: (6610000, 4)
y: (6610000,)

X_train: (3966000, 4)
X_test: (2644000, 4)
y_train: (3966000,)
y_test (2644000,)

X_test: (1322000, 4)
X_val: (1322000, 4)
y_test: (1322000,)
y_val: (1322000,)


In [5]:
with mlflow.start_run():
    mlflow.log_param("model_type", "XGBRegressor")

    model = XGBRegressor(random_state=42)
    model.fit(X_train, y_train)

    print(model)
    print("")

    y_pred_train = model.predict(X_train)
    train_r2 = r2_score(y_true=y_train, y_pred=y_pred_train)
    train_rmse = rmse(y_true=y_train, y_pred=y_pred_train)

    print(f"R2 Score (Training Data): {train_r2}")
    print(f"RMSE (Training Data): {train_rmse}")

    mlflow.log_metric("train_r2", train_r2)
    mlflow.log_metric("train_rmse", train_rmse)

    y_pred_test = model.predict(X_test)
    test_r2 = r2_score(y_true=y_test, y_pred=y_pred_test)
    test_rmse = rmse(y_true=y_test, y_pred=y_pred_test)

    print(f"R2 Score (Test Data): {test_r2}")
    print(f"RMSE (Test Data): {test_rmse}")

    mlflow.log_metric("test_r2", test_r2)
    mlflow.log_metric("test_rmse", test_rmse)

    mlflow.xgboost.log_model(model, "model")

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=None,
             n_jobs=None, num_parallel_tree=None, ...)

R2 Score (Training Data): 0.21278494596481323
RMSE (Training Data): 1.3919494152069092
R2 Score (Test Data): 0.20956403017044067
RMSE (Test Data): 1.3935534954071045


/Users/Work/git/ias_project/ml_training/.venv/lib/python3.12/site-packages/xgboost/sklearn.py:1028: UserWarning: [16:40:12] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  self.get_booster().save_model(fname)
2025/04/13 16:40:14 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


## KNN Regression


In [6]:
from sklearn.neighbors import KNeighborsRegressor

In [7]:
X_train, X_test, X_val, y_train, y_test, y_val = get_data()

Data: (6610000, 5)
Schema({'Time': Int16, 'Num_Cars': Int16, 'Centrality': Float32, 'Is_Entrypoint': Int8, 'Distance': Int16})

X: (6610000, 4)
y: (6610000,)

X_train: (3966000, 4)
X_test: (2644000, 4)
y_train: (3966000,)
y_test (2644000,)

X_test: (1322000, 4)
X_val: (1322000, 4)
y_test: (1322000,)
y_val: (1322000,)


In [8]:
with mlflow.start_run():
    mlflow.log_param("model_type", "KNeighborsRegressor")

    model = KNeighborsRegressor(n_jobs=-1)
    model.fit(X_train, y_train)

    print(model)
    print("")

    y_pred_train = model.predict(X_train)
    train_r2 = r2_score(y_true=y_train, y_pred=y_pred_train)
    train_rmse = rmse(y_true=y_train, y_pred=y_pred_train)

    print(f"R2 Score (Training Data): {train_r2}")
    print(f"RMSE (Training Data): {train_rmse}")

    mlflow.log_metric("train_r2", train_r2)
    mlflow.log_metric("train_rmse", train_rmse)

    y_pred_test = model.predict(X_test)
    test_r2 = r2_score(y_true=y_test, y_pred=y_pred_test)
    test_rmse = rmse(y_true=y_test, y_pred=y_pred_test)

    print(f"R2 Score (Test Data): {test_r2}")
    print(f"RMSE (Test Data): {test_rmse}")

    mlflow.log_metric("test_r2", test_r2)
    mlflow.log_metric("test_rmse", test_rmse)

    mlflow.sklearn.log_model(model, "model")

KNeighborsRegressor(n_jobs=-1)

R2 Score (Training Data): 0.37521500359296933
RMSE (Training Data): 1.2400581420494516
R2 Score (Test Data): 0.09948158281760355
RMSE (Test Data): 1.487430045139285


2025/04/13 16:41:53 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


## Bayesian Linear Regression


In [9]:
from sklearn.linear_model import BayesianRidge

In [10]:
X_train, X_test, X_val, y_train, y_test, y_val = get_data()

Data: (6610000, 5)
Schema({'Time': Int16, 'Num_Cars': Int16, 'Centrality': Float32, 'Is_Entrypoint': Int8, 'Distance': Int16})

X: (6610000, 4)
y: (6610000,)

X_train: (3966000, 4)
X_test: (2644000, 4)
y_train: (3966000,)
y_test (2644000,)

X_test: (1322000, 4)
X_val: (1322000, 4)
y_test: (1322000,)
y_val: (1322000,)


In [11]:
with mlflow.start_run():
    mlflow.log_param("model_type", "BayesianRidge")

    model = BayesianRidge()
    model.fit(X_train, y_train)

    print(model)
    print("")

    y_pred_train = model.predict(X_train)
    train_r2 = r2_score(y_true=y_train, y_pred=y_pred_train)
    train_rmse = rmse(y_true=y_train, y_pred=y_pred_train)

    print(f"R2 Score (Training Data): {train_r2}")
    print(f"RMSE (Training Data): {train_rmse}")

    mlflow.log_metric("train_r2", train_r2)
    mlflow.log_metric("train_rmse", train_rmse)

    y_pred_test = model.predict(X_test)
    test_r2 = r2_score(y_true=y_test, y_pred=y_pred_test)
    test_rmse = rmse(y_true=y_test, y_pred=y_pred_test)

    print(f"R2 Score (Test Data): {test_r2}")
    print(f"RMSE (Test Data): {test_rmse}")

    mlflow.log_metric("test_r2", test_r2)
    mlflow.log_metric("test_rmse", test_rmse)

    mlflow.sklearn.log_model(model, "model")

BayesianRidge()

R2 Score (Training Data): 0.09408408403396606
RMSE (Training Data): 1.4932093620300293
R2 Score (Test Data): 0.09340924024581909
RMSE (Test Data): 1.4924367666244507


2025/04/13 16:42:44 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


## MLP Regression


In [12]:
from sklearn.neural_network import MLPRegressor

In [13]:
X_train, X_test, X_val, y_train, y_test, y_val = get_data()

Data: (6610000, 5)
Schema({'Time': Int16, 'Num_Cars': Int16, 'Centrality': Float32, 'Is_Entrypoint': Int8, 'Distance': Int16})

X: (6610000, 4)
y: (6610000,)

X_train: (3966000, 4)
X_test: (2644000, 4)
y_train: (3966000,)
y_test (2644000,)

X_test: (1322000, 4)
X_val: (1322000, 4)
y_test: (1322000,)
y_val: (1322000,)


In [14]:
with mlflow.start_run():
    mlflow.log_param("model_type", "MLPRegressor")

    model = MLPRegressor(random_state=42, verbose=True)
    model.fit(X_train, y_train)

    print(model)
    print("")

    y_pred_train = model.predict(X_train)
    train_r2 = r2_score(y_true=y_train, y_pred=y_pred_train)
    train_rmse = rmse(y_true=y_train, y_pred=y_pred_train)

    print(f"R2 Score (Training Data): {train_r2}")
    print(f"RMSE (Training Data): {train_rmse}")

    mlflow.log_metric("train_r2", train_r2)
    mlflow.log_metric("train_rmse", train_rmse)

    y_pred_test = model.predict(X_test)
    test_r2 = r2_score(y_true=y_test, y_pred=y_pred_test)
    test_rmse = rmse(y_true=y_test, y_pred=y_pred_test)

    print(f"R2 Score (Test Data): {test_r2}")
    print(f"RMSE (Test Data): {test_rmse}")

    mlflow.log_metric("test_r2", test_r2)
    mlflow.log_metric("test_rmse", test_rmse)

    mlflow.sklearn.log_model(model, "model")

Iteration 1, loss = 1.04002500
Iteration 2, loss = 1.03235185
Iteration 3, loss = 1.02938068
Iteration 4, loss = 1.02779078
Iteration 5, loss = 1.02649188
Iteration 6, loss = 1.02544248
Iteration 7, loss = 1.02457106
Iteration 8, loss = 1.02347517
Iteration 9, loss = 1.02260268
Iteration 10, loss = 1.02186048
Iteration 11, loss = 1.02130949
Iteration 12, loss = 1.02098083
Iteration 13, loss = 1.02094650
Iteration 14, loss = 1.02058840
Iteration 15, loss = 1.02036154
Iteration 16, loss = 1.02026320
Iteration 17, loss = 1.02010155
Iteration 18, loss = 1.01990640
Iteration 19, loss = 1.01977825
Iteration 20, loss = 1.01974905
Iteration 21, loss = 1.01959395
Iteration 22, loss = 1.01933146
Iteration 23, loss = 1.01891053
Iteration 24, loss = 1.01834548
Iteration 25, loss = 1.01804018
Iteration 26, loss = 1.01783276
Iteration 27, loss = 1.01754177
Iteration 28, loss = 1.01745856
Iteration 29, loss = 1.01734650
Iteration 30, loss = 1.01707852
Iteration 31, loss = 1.01704168
Iteration 32, los

2025/04/13 16:47:28 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


## Tensorflow Dense Neural Net


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import RootMeanSquaredError as RMSE, R2Score
import mlflow.tensorflow
import mlflow.keras

In [16]:
X_train, X_test, X_val, y_train, y_test, y_val = get_data()


Data: (6610000, 5)
Schema({'Time': Int16, 'Num_Cars': Int16, 'Centrality': Float32, 'Is_Entrypoint': Int8, 'Distance': Int16})

X: (6610000, 4)
y: (6610000,)

X_train: (3966000, 4)
X_test: (2644000, 4)
y_train: (3966000,)
y_test (2644000,)

X_test: (1322000, 4)
X_val: (1322000, 4)
y_test: (1322000,)
y_val: (1322000,)


In [17]:
OPTIMIZER = "Adam"
LR = 0.001
EPOCHS = 10
BATCH_SIZE = 32

with mlflow.start_run():
    model = Sequential(
        [
            Dense(128, activation="relu"),
            Dense(64, activation="relu"),
            Dense(32, activation="relu"),
            Dense(1),
        ]
    )
    model.build(input_shape=(None, X_train.shape[1]))

    model.compile(
        optimizer=Adam(learning_rate=LR), loss="mse", metrics=[RMSE(), R2Score()]
    )

    mlflow.log_param("model_type", "TensorFlow DNN")
    mlflow.log_param("optimizer", OPTIMIZER)
    mlflow.log_param("epochs", EPOCHS)
    mlflow.log_param("batch_size", BATCH_SIZE)

    history = model.fit(
        X_train,
        y_train,
        validation_data=(X_val, y_val),
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        verbose=1,
    )

    for epoch in range(EPOCHS):
        mlflow.log_metric("train_loss", history.history["loss"][epoch], step=epoch)
        mlflow.log_metric(
            "train_rmse", history.history["root_mean_squared_error"][epoch], step=epoch
        )
        mlflow.log_metric("train_r2", history.history["r2_score"][epoch], step=epoch)
        mlflow.log_metric("val_loss", history.history["val_loss"][epoch], step=epoch)
        mlflow.log_metric(
            "val_rmse",
            history.history["val_root_mean_squared_error"][epoch],
            step=epoch,
        )
        mlflow.log_metric("val_r2", history.history["val_r2_score"][epoch], step=epoch)

    loss, rmse, r2_score = model.evaluate(X_test, y_test, verbose=0)

    mlflow.log_metric("test_loss", loss)
    mlflow.log_metric("test_rmse", rmse)
    mlflow.log_metric("test_r2", r2_score)

    mlflow.keras.log_model(model, "model")

    print("")
    print(f"Loss: {loss}")
    print(f"RMSE: {rmse}")
    print(f"R2Score: {r2_score}")

Epoch 1/10
123938/123938 ━━━━━━━━━━━━━━━━━━━━ 63s 504us/step - loss: 2.0662 - r2_score: 0.1580 - root_mean_squared_error: 1.4374 - val_loss: 2.0330 - val_r2_score: 0.1711 - val_root_mean_squared_error: 1.4258
Epoch 2/10
123938/123938 ━━━━━━━━━━━━━━━━━━━━ 60s 486us/step - loss: 2.0331 - r2_score: 0.1731 - root_mean_squared_error: 1.4259 - val_loss: 2.0241 - val_r2_score: 0.1747 - val_root_mean_squared_error: 1.4227
Epoch 3/10
123938/123938 ━━━━━━━━━━━━━━━━━━━━ 61s 489us/step - loss: 2.0266 - r2_score: 0.1755 - root_mean_squared_error: 1.4236 - val_loss: 2.0169 - val_r2_score: 0.1777 - val_root_mean_squared_error: 1.4202
Epoch 4/10
123938/123938 ━━━━━━━━━━━━━━━━━━━━ 60s 487us/step - loss: 2.0208 - r2_score: 0.1770 - root_mean_squared_error: 1.4215 - val_loss: 2.0187 - val_r2_score: 0.1769 - val_root_mean_squared_error: 1.4208
Epoch 5/10
123938/123938 ━━━━━━━━━━━━━━━━━━━━ 61s 489us/step - loss: 2.0185 - r2_score: 0.1785 - root_mean_squared_error: 1.4207 - val_loss: 2.0208 - val_r2_score: 

2025/04/13 17:02:56 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2025/04/13 17:03:01 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.



Loss: 2.0134222507476807
RMSE: 1.418951153755188
R2Score: 0.18048977851867676
